In [ ]:
# Visualize the scores for a selected number of selective search proposals.
# These scores have been computed beforehand and are stored in a file.

In [ ]:
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models

from tqdm import tqdm
from pathlib import Path
from PIL import Image

from arid import arid
import json

def map_boxes(box):
    x1, y1, x2, y2 = box
    return ((x1, y1), (x2, y1), (x2, y2), (x1, y2))

wps = arid.get_wps("/home/justin/Desktop/arid-dataset")
ss_root = Path('/home/justin/Desktop/thesis/ss')

In [ ]:
num_imgs = 0
num_wps = 0

OBJ = 'apple_1'
img_choices = []
for wp in wps:
    imgs = wp.get_images_with_object(OBJ)
    if imgs:
        num_wps += 1
        img_ids = list(map(lambda x: x['path'].stem, imgs))
        num_imgs += len(img_ids)
        print(f'{wp.get_title()}-{"|".join(img_ids)}')
        img_choices.append(wp.get_title())
        
print(num_imgs)
print(num_wps)

In [ ]:
# Top 5
import random
title = random.choice(img_choices)
_wps = {_wp.get_title(): _wp for _wp in wps}
print(title)
wp = _wps[title]
data = Path(f'/home/justin/Desktop/thesis-results/{OBJ}-{title}.json')
with open(data) as f:
    rank_data = json.load(f)

ss_path = Path(ss_root /'ss-fast'/ f'ss-fast-{title}.json')
with open(ss_path) as f:
    ss_data = json.load(f)

img_paths = wp.rgb_image_paths()

for img_path in img_paths:
    new_img_path = arid.annotation_path(img_path, 'selective-search')
    img_id = Path(img_path).stem
    if rank_data.get(img_id) is None:
        continue
    img = Image.open(img_path)
    print(img_id)
    ss_boxes = ss_data[img_id]['boxes']
    scores = rank_data[img_id]

    z = list(zip(ss_boxes, scores))
    sorted_scores = sorted(z, key=lambda s:s[1], reverse=True)
    print(len(scores))
    
    CUTOFF = 100
    hi_scores = sorted_scores[:CUTOFF]
    taken = [False] * CUTOFF
    
    acc = []
    for idx, (box, score) in enumerate(hi_scores):
        x1, y1, x2, y2 = box
        
        for jdx in range(idx+1, len(hi_scores)):
            iou = arid.compute_bbox_iou(map_boxes(box), map_boxes(hi_scores[jdx][0]))
            if iou > 0.1:
                taken[jdx] = True
    
        if not taken[idx]:
            acc.append({
                'id': f'{str(idx)}',
                'coords': ((x1, y1), (x2, y1), (x2, y2), (x1, y2)),
                'score': 0.99 - (idx * 0.05) ,
                'colormap': 'spring_r'
            })
            if len(acc) == 5:
                break
    arid.annotate_img(img, new_img_path, acc, save=False)
    display(img)


In [ ]:
# Over threshold
num_imgs = 0
num_wps = 0

OBJ = 'tomato_0'
img_choices = []
for wp in wps:
    imgs = wp.get_images_with_object(OBJ)
    if imgs:
        num_wps += 1
        img_ids = list(map(lambda x: x['path'].stem, imgs))
        num_imgs += len(img_ids)
        img_choices.append(wp.get_title())
        
import random
title = random.choice(img_choices)
_wps = {_wp.get_title(): _wp for _wp in wps}
print(title)
wp = _wps[title]
data = Path(f'/home/justin/Desktop/thesis-results/{OBJ}-{title}.json')
with open(data) as f:
    rank_data = json.load(f)

ss_path = Path(ss_root /'ss-fast'/ f'ss-fast-{title}.json')
with open(ss_path) as f:
    ss_data = json.load(f)

img_paths = wp.rgb_image_paths()

for img_path in img_paths:
    new_img_path = arid.annotation_path(img_path, 'selective-search')
    img_id = Path(img_path).stem
    if rank_data.get(img_id) is None:
        continue
    img = Image.open(img_path)
    print(img_id)
    ss_boxes = ss_data[img_id]['boxes']
    scores = rank_data[img_id]

    z = list(zip(ss_boxes, scores))
    sorted_scores = sorted(z, key=lambda s:s[1], reverse=True)
    print(len(scores))

    acc = []
    for idx, (box, score) in enumerate(sorted_scores):
        x1, y1, x2, y2 = box
        if idx < 10:
            acc.append({
                'id': f'{str(round(score,2))}',
                'coords': ((x1, y1), (x2, y1), (x2, y2), (x1, y2)),
                'score': 0.99 - (idx * 0.15),
                'colormap': 'spring_r'
            })

    arid.annotate_img(img, new_img_path, acc, save=False)
    display(img)


In [ ]:
#View Ranking
title = 'wp_7_2_2'
_wps = {_wp.get_title(): _wp for _wp in wps}
print(title)
wp = _wps[title]
data = Path(f'/home/justin/Desktop/thesis-results/{OBJ}-{title}.json')
with open(data) as f:
    rank_data = json.load(f)

ss_path = Path(ss_root /'ss-fast'/ f'ss-fast-{title}.json')
with open(ss_path) as f:
    ss_data = json.load(f)

img_paths = wp.rgb_image_paths()
for img_p in img_paths:
    if img_p.stem == '013':
        img_path = img_p

new_img_path = arid.annotation_path(img_path, 'selective-search')
img_id = Path(img_path).stem

img = Image.open(img_path)
print(img_id)
ss_boxes = ss_data[img_id]['boxes']
scores = rank_data[img_id]

z = list(zip(ss_boxes, scores))
sorted_scores = sorted(z, key=lambda s:s[1], reverse=True)

acc = []
a = False
b = False
c = False
d = False
e = False

f = False
g = False
h = False
i = False
j = False

for idx, (box, score) in enumerate(sorted_scores):
    x1, y1, x2, y2 = box
    app = False
    if not a:
        a = True
        app = True
    elif not b and score < 0.9:
        b = True
        app = True
    elif not c and score < 0.8:
        c = True
        app = True
    elif not d and score < 0.7:
        d = True
        app = True
    elif not e and score < 0.6:
        e = True
        app = True
    elif not f and score < 0.5:
        f = True
        app = True
    elif not g and score < 0.4:
        g = True
        app = True
    elif not h and score < 0.3:
        h = True
        app = True
    elif not i and score < 0.2:
        i = True
        app = True   
    elif not j and score < 0.1:
        j = True
        app = True
    
    if app:
        acc.append({
            'id': f'{round(score, 5)}',
            'coords': ((x1, y1), (x2, y1), (x2, y2), (x1, y2)),
            'score': score,
            'colormap': 'spring_r'
        })
        i2 = img.crop((x1, y1, x2, y2))
        display(i2)
acc.append({
    'id': f'{round(score, 5)}',
    'coords': ((x1, y1), (x2, y1), (x2, y2), (x1, y2)),
    'score': score,
    'colormap': 'spring_r'
})
i2 = img.crop((x1, y1, x2, y2))
display(i2)

arid.annotate_img(img, new_img_path, acc, save=False)
display(img)


In [ ]:
#View Ranking
_wps = {_wp.get_title(): _wp for _wp in wps}
title = random.choice(list(_wps.keys()))
print(title)
wp = _wps[title]
data = Path(f'/home/justin/Desktop/thesis-results/{OBJ}-{title}.json')
with open(data) as f:
    rank_data = json.load(f)

ss_path = Path(ss_root /'ss-fast'/ f'ss-fast-{title}.json')
with open(ss_path) as f:
    ss_data = json.load(f)

img_paths = wp.rgb_image_paths()
for img_p in img_paths:
    if img_p.stem == '013':
        img_path = img_p

new_img_path = arid.annotation_path(img_path, 'selective-search')
img_id = Path(img_path).stem

img = Image.open(img_path)
print(img_id)
ss_boxes = ss_data[img_id]['boxes']
scores = rank_data[img_id]

z = list(zip(ss_boxes, scores))
sorted_scores = sorted(z, key=lambda s:s[1], reverse=True)

acc = []
for idx, (box, score) in enumerate(sorted_scores):
    x1, y1, x2, y2 = box
    
    if score > 0.98:
        acc.append({
            'id': f'{round(score, 5)}',
            'coords': ((x1, y1), (x2, y1), (x2, y2), (x1, y2)),
            'score': score,
            'colormap': 'spring_r'
        })

arid.annotate_img(img, new_img_path, acc, save=False)
display(img)
